In [1]:
import numpy as np
from PIL import Image

In [2]:
input_image = Image.open("fruit_crop.png")
np_input_image = np.array(input_image, dtype=np.float32)
print("Input image shape: ", np_input_image.shape, "Input image dtype: ", np_input_image.dtype)

Input image shape:  (2048, 2048, 3) Input image dtype:  float32


In [3]:
bayer_pattern = np.array([[[0.,1.], [0.,0.]], 
                          [[1.,0.], [0.,1.]], 
                          [[0.,0.], [1.,0.]]]).transpose(1, 2, 0).astype(np.float32)
pil_bayer_pattern = Image.fromarray((bayer_pattern * 255).astype(np.uint8))
pil_bayer_pattern.save("bayer_pattern.jpg")
print("Bayer Pattern Shape:", bayer_pattern.shape, "Bayer Pattern Dtype:",bayer_pattern.dtype)

Bayer Pattern Shape: (2, 2, 3) Bayer Pattern Dtype: float32


In [4]:
def mosaicking(input_image, bayer):
    output_image = np.zeros_like(input_image)
    for i in range(0, input_image.shape[0], 2):
        for j in range(0, input_image.shape[1], 2):
            output_image[i:i+2, j:j+2] = input_image[i:i+2, j:j+2] * bayer
    return output_image

In [5]:
def padding(mosaic_image):
    padded_image = np.zeros((mosaic_image.shape[0] + 2, mosaic_image.shape[1] + 2, mosaic_image.shape[2]), dtype=mosaic_image.dtype)
    padded_image[1:-1, 1:-1, :] = mosaic_image
    padded_image[0, 1:-1] = mosaic_image[1, :]
    padded_image[-1, 1:-1] = mosaic_image[-2, :]
    padded_image[1:-1, 0] = mosaic_image[:, 1]
    padded_image[1:-1, -1] = mosaic_image[:, -2]
    padded_image[0, 0] = mosaic_image[1, 1]
    padded_image[0, -1] = mosaic_image[1, -2]
    padded_image[-1, 0] = mosaic_image[-2, 1]
    padded_image[-1, -1] = mosaic_image[-2, -2]

    return padded_image

In [6]:
np_mosaic_image = mosaicking(np_input_image, bayer_pattern)
print("Mosaic Image Shape:", np_mosaic_image.shape, "Mosaic Image Dtype:",np_mosaic_image.dtype)
print("Input:", np_input_image[1][0], "Output:", np_mosaic_image[1][0])

pil_mosaic_image = Image.fromarray(np_mosaic_image.astype(np.uint8))
pil_mosaic_image.save("mosaic_image.png")

np_padded_image = padding(np_mosaic_image)
pil_padded_image = Image.fromarray(np_padded_image.astype(np.uint8))
pil_padded_image.save("padded_image.png")

Mosaic Image Shape: (2048, 2048, 3) Mosaic Image Dtype: float32
Input: [252. 255. 226.] Output: [  0.   0. 226.]


In [9]:
output_image = np_padded_image.copy()

for i in range(0, 3):
    for j in range(0, 3):
        if np_padded_image[i, j, 0] == 0:
            print("Kırmızı Kanal")
            red_channel = np_padded_image[i:i+3, j:j+3,0]
            red_channel_avg = np.sum(red_channel) / np.count_nonzero(red_channel)
            output_image[i+1, j+1, 0] = red_channel_avg
        
        if np_padded_image[i, j, 1] == 0:
            print("Yeşil Kanal")
            green_channel = np_padded_image[i:i+3, j:j+3,1]
            green_channel_avg = np.sum(green_channel) / np.count_nonzero(green_channel)
            output_image[i+1, j+1, 1] = green_channel_avg

        if np_padded_image[i, j, 2] == 0:
            print("Mavi Kanal")
            blue_channel = np_padded_image[i:i+3, j:j+3,2]
            blue_channel_avg = np.sum(blue_channel) / np.count_nonzero(blue_channel)
            output_image[i+1, j+1, 2] = blue_channel_avg

        print("np_padded_image:", np_padded_image[i,j])
        print("output_image:", output_image[i+1, j+1])

        
        break
    break

Kırmızı Kanal
Mavi Kanal
np_padded_image: [  0. 255.   0.]
output_image: [252. 255. 226.]
